**Nama : Fenny Feronika Maharani**

**NIM  : 1703015**

**Tugas Klasifikasi dengan Deep Learning**

Library yang digunakan:

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.models import load_model
from keras import optimizers
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import os
import random as rn

**1. LOAD DATA SET**

  Karena format file dari data set merupakan .txt maka kita menggunakan fungsi open biasa. Dengan melakukan pemisahan antara text dan label yang dipisahkan oleh tanda "==>". Setiap pasangan text dan label dipisahkan dengan enter atau "\n". Dalam data set sendiri selalu terdapat whitespace di awal dan akhir text dan label maka whitespace itu dapat dihilangkan menggunakan .strip()(untuk menghilangkan whitespace di awal dan akhir)

In [ ]:
namaFile = "/content/drive/My Drive/KULIAH/smt6/NLP/KLASIFIKASI/dataset_klasifikasi_review_restoran.txt"
with open(namaFile, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
data = []
label = []
for line in lines:
     txt = line.split('==>')
     if len(txt) != 1 :
         data.append(txt[0].strip())
         label.append(txt[1].strip())

print(data[:5])
print(label[:5])

['Tempat ini mengimingi makan steak dan daging dengan harga murah . Namun kenyataannya tidak murah jika dibandingkan dengan kualitas dagingnya yang tidak enak . Dagingnya keras dan rasanya hambar .', "Sudah lama sekali tidak mengunjungi resto yang satu ini . Tahu goreng berbumbunya enak . Gurame ma ' keke nya manis pedas kurang berapa enak . Nasi ulamnya komplit dan murah .", 'Kunjungan ke restoran ini sangat mengecewakan karena dua menu yg saya pesan tidak tersedia seharusnya saat menyajikan menu hal ini disampaikan ke pembeli', 'Pertama kali nyobain makan disini , karena reviewnya sob buntutnya terkenal enak banget . Padahal gak terlalu doyan sop buntut sih sebenernya , tapi mumpung lagi di bandung ya udah cobain mampir aja . Kayanya enak makan sop hangat di udara dingin bandung . Tempatnya rumahan banget , homey dan nyaman . Meski kita datang gak pas jam makan , tapi tetep rame pengunjung .', 'Saya bersama keluarga makan di dapur dahapati anggrek . Semuanya baik2 saja , dari rasa ma

**2. KONVERSI LABEL MENJADI REPRESENTASI TENSOR**

Konversi label dari "NEG", "NETRAL", "POS" menjadi representasi tensor.

Hasilnya adalah label tensor 2D dengan shape (992, 3), karena ada 992 instance dengan 3 nilai label yang mungkin (negatif, netral, positif)



In [ ]:
code = np.array(label)
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(code)
label = to_categorical(vec)
print(label.shape)
print(label)

(992, 3)
[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


**3. SPLIT DATASET**

Split data menjadi train dan test menggunakan scikit learn, 80% menjadi data train, 20% menjadi data test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, label,
                                                   test_size=0.2,
                                                   random_state=123)

**4. KONVERSI DATA TEKS MENJADI TF-IDF DAN TENSOR**

Hasilnya adalah tensor 2D (793, 5646) untuk data train dan tensor 2D (119, 5646) untuk data test.

In [ ]:
#konversi teks ke tfidf
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
#fit hanya berdasarkan data train
tokenizer.fit_on_texts(X_train)
#konversi train
seq_x_train = tokenizer.texts_to_sequences(X_train)
X_enc_train = tokenizer.sequences_to_matrix(seq_x_train,mode="tfidf")
#konversi test
seq_x_test  = tokenizer.texts_to_sequences(X_test)
X_enc_test  = tokenizer.sequences_to_matrix(seq_x_test,mode="tfidf")

print(X_enc_train.shape)
print(X_enc_test.shape)
print(X_enc_train)

(793, 5646)
(199, 5646)
[[0.         1.50791899 1.71312898 ... 0.         0.         0.        ]
 [0.         0.         1.01180157 ... 0.         0.         0.        ]
 [0.         0.89060125 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.50791899 1.01180157 ... 0.         0.         5.98519493]
 [0.         0.         1.71312898 ... 0.         0.         0.        ]]


**5. Persiapan Model**

Sebelum melakukan Training kita harus mempersiapkan dahulu model

Ada empat layer: layer pertama adalah layer input hasil encode tf-idf sebelumnya: 5646 fitur. Mengapa  input_shape tidak menggunakan sample dimension atau sample axis  seperti (793, 5646)? karena jumlah samples tidak penting didefinisikan dalam layer input. Dengan teknik mini-batch,  sample dapat  diproses sedikit demi sedikit, jadi jumlahnya bisa berbeda-beda.  

Activation softmax digunakan karena jumlah label ada 3 (negatif, netral dan positif). Jika jumlah label dua (binary classification) maka dapat digunakan activation sigmoid.   Setelah layer didefinisikan, maka layer dapat dicompile. Loss  categorical_crossentropy  dipilih karena terdapat tiga kelas,  sedangkan jika untuk binary class dapat digunakan  binary_crossentropy.


# Proses  mencari hyperparameter terbaik, dengan percobaan mengganti:

1. Jumlah neuron

    Percobaan mengganti jumlah neuron pada layer tidak mempengaruhi waktu training, dari seluruh percobaan dimulai dari 
    
    a. Layer peryama 32 dan layer kedua 4

    b. Layer peryama 16 dan layer kedua 8

    c. Layer peryama 64 dan layer kedua 2

    d. Layer peryama 8, layer kedua 6 dan layer ketiga 4

* Akurasi tertinggi didapat oleh poin c
* Grafik yang mendekati good fit dihasilkan oleh poin a

In [ ]:
_,jum_fitur = X_enc_train.shape
model = models.Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(jum_fitur,)))
model.add(layers.Dense(4,activation='relu'))
model.add(layers.Dense(3,activation='softmax'))  #karena kelasnya ada 3

2. Fungsi Aktivasi

    Percobaan mengganti fugsi aktivasi dari relu menjadi tanh berdampak pada grafik loss jika dibandingkan dengan relu, tanh membuat grafik menjadi overfit namun akurasi meningkat

In [ ]:
model = models.Sequential()
model.add(layers.Dense(32, activation='tanh',input_shape=(jum_fitur,)))
model.add(layers.Dense(4, activation='tanh'))
model.add(layers.Dense(3, activation='softmax')) 

  3. Optimizer

    Dari percobaan mengganti optimizer Adam, Adagrad, Adamax, SGD menghasilkan:
*   Akurasi tertinggi didapat oleh opt Adam 
*   Grafik yang mendekati good fit didapat oleh opt Adam

![Grafik Loss Adam](https://drive.google.com/file/d/11BgW8rfibs4KV4nmYc2zUIewkewqxZMZ/view?usp=sharing)

In [ ]:
model.compile(optimizer="adam",
             loss='categorical_crossentropy',
             metrics=['accuracy'])

4. Learning Rate

    Percobaan mengganti learning rate pada opt adam menjadi 0,01 dan 0,0001 mengakibatkan turunnya akurasi dan grafik menjadi overfitting.

In [ ]:
opt = optimizers.Adam(lr=0.0001)
model.compile(optimizer= opt,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

Pada neural network, nilai random digunakan untuk inisialisasi bobot dan split batch. Ini menyebabkan konfigurasi yang sama dapat memberikan hasil berbeda setiap dijalankan. Untuk mencegahnya maka di perlukan kode seperti dibawah ini

In [ ]:
tf.random.set_seed(123)
np.random.RandomState(123)  
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
rn.seed(123)

**6. PROSES TRAINING**



Proses  mencari hyperparameter terbaik, selain dengan percobaan diatas dapat juga mengganti:
5. Jumlah epoch

    Dengan menambah jumlah epoch dari 3 menjadi 10, 20, 30, dan 50. Akurasi yang didapat memang semakin tinggi, namun dilihat dari grafik epoch vs loss terjadi overfitting. Jumlah epoch yang terlalu banyak membuat proses train terlalu lama dan model terlalu overfit terhadap data validasi, tetapi jika terlalu sedikit, model mungkin saja belum konvergen. Maka lebih baik training dilakukan dengan menggunakan early stopping. Keras menyediakan fasilitas callback yang dapat dimanfaatkan untuk menghentikan proses training jika sudah memenuhi kriteria tertentu.   

In [ ]:
history = model.fit(X_enc_train,y_train,
                   epochs=20, batch_size=2, 
                   validation_split=0.2)
results = model.evaluate(X_enc_test, y_test) 
print("Hasil  [loss,acc] untuk data test:")
print(results)

Train on 634 samples, validate on 159 samples
Epoch 1/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0153 - accuracy: 0.9937 - val_loss: 0.7198 - val_accuracy: 0.7547
Epoch 2/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0109 - accuracy: 0.9953 - val_loss: 0.7526 - val_accuracy: 0.7484
Epoch 3/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0081 - accuracy: 0.9968 - val_loss: 0.7742 - val_accuracy: 0.7421
Epoch 4/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0070 - accuracy: 0.9968 - val_loss: 0.7876 - val_accuracy: 0.7421
Epoch 5/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0064 - accuracy: 0.9968 - val_loss: 0.8033 - val_accuracy: 0.7421
Epoch 6/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0060 - accuracy: 0.9968 - val_loss: 0.8205 - val_accuracy: 0.7484
Epoch 7/20
634/634 [==============================] - 1s 1ms/step - loss: 0.0057 - accuracy: 0.9968 - 

Menggunakan Early Stopping dengan syarat dalam proses train jika nilai loss pada data validation (val_loss) tidak berubah selama 3 epoch (patience=3). 


In [ ]:
esCallBack = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=0,
                                              patience=3,
                                              verbose=0, mode='auto')
history = model.fit(X_enc_train,y_train,
                  epochs=20, batch_size=2,
                  validation_split=0.2, callbacks=[esCallBack])

results = model.evaluate(X_enc_test, y_test) 
print("Hasil  [loss,acc] untuk data test:")
print(results)

Train on 634 samples, validate on 159 samples
Epoch 1/20
634/634 [==============================] - 1s 2ms/step - loss: 0.9098 - accuracy: 0.6293 - val_loss: 0.6579 - val_accuracy: 0.7610
Epoch 2/20
634/634 [==============================] - 1s 2ms/step - loss: 0.3559 - accuracy: 0.8817 - val_loss: 0.5689 - val_accuracy: 0.7736
Epoch 3/20
634/634 [==============================] - 1s 2ms/step - loss: 0.1212 - accuracy: 0.9669 - val_loss: 0.7103 - val_accuracy: 0.7421
Epoch 4/20
634/634 [==============================] - 1s 2ms/step - loss: 0.0442 - accuracy: 0.9905 - val_loss: 0.7102 - val_accuracy: 0.7233
Epoch 5/20
199/199 [==============================] - 0s 76us/step
Hasil  [loss,acc] untuk data test:
[1.185609549134221, 0.6934673190116882]


6. Batch size

    Dengan menambah batch size proses training menjadi lebih cepat, akurasi yang didapat malah menurun.


In [ ]:
esCallBack = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=0,
                                              patience=3,
                                              verbose=0, mode='auto')
history = model.fit(X_enc_train,y_train,
                  epochs=20, batch_size=100,
                  validation_split=0.2, callbacks=[esCallBack])

results = model.evaluate(X_enc_test, y_test) 
print("Hasil  [loss,acc] untuk data test:")
print(results)

Train on 634 samples, validate on 159 samples
Epoch 1/20
634/634 [==============================] - 0s 108us/step - loss: 0.0104 - accuracy: 0.9984 - val_loss: 0.8801 - val_accuracy: 0.7987
Epoch 2/20
634/634 [==============================] - 0s 98us/step - loss: 0.0104 - accuracy: 0.9984 - val_loss: 0.8808 - val_accuracy: 0.7987
Epoch 3/20
634/634 [==============================] - 0s 90us/step - loss: 0.0104 - accuracy: 0.9984 - val_loss: 0.8816 - val_accuracy: 0.7987
Epoch 4/20
199/199 [==============================] - 0s 76us/step
Hasil  [loss,acc] untuk data test:
[1.740022554469468, 0.6834170818328857]


**7. MENYIMPAN MODEL DAN HASIL TOKENIZER**

In [ ]:
import pickle
model.save('model_spam_v1.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


**8. ME-LOAD MODEL**

In [ ]:
model = load_model('model_spam_v1.h5')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

s  = ["makanannya enak tempat nyaman.", "pelayanan buruk, harga sangat mahal, tempat tidak nyaman."]
seq_str = tokenizer.texts_to_sequences(s)
enc_str = tokenizer.sequences_to_matrix(seq_str,mode="tfidf")
enc_str.shape
pred = model.predict_classes(enc_str)
print("Prediksi kelas string ' {} ' \nadalah {}".format(s,pred))


Prediksi kelas string ' ['makanannya enak tempat nyaman.', 'pelayanan buruk, harga sangat mahal, tempat tidak nyaman.'] ' 
adalah [2 0]


**9. PERHITUNGAN AKURASI DAN F1**

In [ ]:
y_pred = model.predict_classes(X_enc_test)
print(classification_report(np.argmax(y_test,axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.56      0.81      0.66        59
           1       0.44      0.11      0.18        35
           2       0.83      0.82      0.82       105

    accuracy                           0.69       199
   macro avg       0.61      0.58      0.56       199
weighted avg       0.68      0.69      0.66       199

